In [6]:
# Uncomment this line if you need to install the dependencies
%pip install folium httpx -q
%pip install rio-tiler titiler -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import json
import httpx
import pprint as pp

from folium import Map, TileLayer

In [8]:
titiler_endpoint = "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.

# one of the SLGA layers:
url = "https://esoil.io/TERNLandscapes/Public/Products/TERN/SLGA/30m/SOC/SOC_000_005_EV_N_P_AU_TRN_N_20220727_30m.tif"

## Get COG Info

In [9]:
# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
pp.pp(r)

{'bounds': [112.999861111,
            -44.00013888899189,
            153.99986111200673,
            -10.000138888],
 'minzoom': 2,
 'maxzoom': 12,
 'band_metadata': [['b1',
                    {'STATISTICS_MAXIMUM': '17.097089767456',
                     'STATISTICS_MEAN': '1.0913863631295',
                     'STATISTICS_MINIMUM': '0.0068007996305823',
                     'STATISTICS_STDDEV': '1.1193720895427',
                     'STATISTICS_VALID_PERCENT': '49.31'}]],
 'band_descriptions': [['b1', '']],
 'dtype': 'float32',
 'nodata_type': 'Nodata',
 'colorinterp': ['gray'],
 'scales': [1.0],
 'offsets': [0.0],
 'driver': 'GTiff',
 'count': 1,
 'width': 147600,
 'height': 122400,
 'overviews': [2, 4, 8, 16, 32, 64, 128, 256, 511, 1018],
 'nodata_value': None}


### Display Tiles

In [10]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=13
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="SLGA Soil Carbon 30m"
).add_to(m)

m

# Work with non-byte data

In [14]:

# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

pp.pp(r)
print("Data is of type:", r["dtype"])

# This dataset has statistics metadata
minv, maxv = r["band_metadata"][0][1]["STATISTICS_MINIMUM"], r["band_metadata"][0][1]["STATISTICS_MAXIMUM"]
print("With values from ", minv, "to ", maxv)



{'bounds': [112.999861111,
            -44.00013888899189,
            153.99986111200673,
            -10.000138888],
 'minzoom': 2,
 'maxzoom': 12,
 'band_metadata': [['b1',
                    {'STATISTICS_MAXIMUM': '17.097089767456',
                     'STATISTICS_MEAN': '1.0913863631295',
                     'STATISTICS_MINIMUM': '0.0068007996305823',
                     'STATISTICS_STDDEV': '1.1193720895427',
                     'STATISTICS_VALID_PERCENT': '49.31'}]],
 'band_descriptions': [['b1', '']],
 'dtype': 'float32',
 'nodata_type': 'Nodata',
 'colorinterp': ['gray'],
 'scales': [1.0],
 'offsets': [0.0],
 'driver': 'GTiff',
 'count': 1,
 'width': 147600,
 'height': 122400,
 'overviews': [2, 4, 8, 16, 32, 64, 128, 256, 511, 1018],
 'nodata_value': None}
Data is of type: float32
With values from  0.0068007996305823 to  17.097089767456


### Display Tiles


Note: By default if the metadata has `min/max` statistics, titiler will use those to rescale the data

In [15]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=5
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Swisstopo"
).add_to(m)
m

Apply ColorMap

Now that the data is rescaled to byte values (0 -> 255) we can apply a colormap

In [16]:
r = httpx.get(
    f"{titiler_endpoint}/cog/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        "rescale": f"{minv},{maxv}",
        "colormap_name": "terrain"
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=5
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Swisstopo"
).add_to(m)
m